# Student name, ID : Morteza Layegh Mirhosseini (220003166) - PG

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
    import torch
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

cpu


# TO DO (IMPORTANT)

In [ ]:
import os
# TODO: Fill in the Google Drive path where you uploaded the shared folder
# Example: GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'coursework'

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'NLP/coursework' # Change this line
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

#you should see the following in the out put: 
#['coursework .ipynb', 'trained_models', 'dataset', 'feature_models', 'Test .ipynb', 'README.txt']

['Test .ipynb', 'trained_models', 'dataset', 'feature_models', 'README.txt', 'coursework .ipynb']


# Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score ,accuracy_score
from sklearn.svm import SVC
import spacy
import gensim
import re
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import os
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

#from datasets import load_dataset
from gensim.models import Word2Vec, KeyedVectors

import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install transformers
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pickle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.1 MB/s eta 0:00:00


# Pre-Processing Function


In [ ]:


def preprocess_text(text):
    
    text = text.lower()
    
    text = re.sub(r'http\S+', '', text)
    
    text = re.sub(r'\S+@\S+', '', text)
    
    text = re.sub(r'([aeiou])\1{2,}', r'\1', text)
    
    # Replace negative constructs with "not"
    text = re.sub(r"can\'t|cannot", "can not", text)
    text = re.sub(r"don\'t|doesn\'t", "do not", text)
    text = re.sub(r"won\'t|wouldn\'t", "will not", text)
    text = re.sub(r"shouldn\'t|couldn\'t", "should not", text)
    text = re.sub(r"haven\'t|hasn\'t", "have not", text)
    text = re.sub(r"hadn\'t|wasn\'t|weren\'t|isn\'t|aren\'t", "not", text)
    text = re.sub(r"never|none|nothing|nowhere|n\'t", "not", text)
    
    # Replace emoticons with "positive" or "negative"
    text = re.sub(r"(:\)|:-\)|:\]|:-\])+", "positive", text)
    text = re.sub(r"(:\(|:<|\[|\]-\(|:-\[|:-\(|:-{|:-/|:-\\)+", "negative", text)
    
    # punctuations and whitespace characters
    #stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    words = [word for word in words if word.isalnum()]


    text = ' '.join(words)

    return text


from sklearn.feature_extraction.text import TfidfVectorizer
# function is adapted from the City NLP Lab (Text Classification Advanced)

def custom_feature_extraction(text, ngrams=2, compute=0, vectorizer=None):
    if compute:
        # Extract unigrams, bigrams, and trigrams
        vectorizer = TfidfVectorizer(ngram_range=(1, ngrams),preprocessor= preprocess_text)
        features = vectorizer.fit_transform(text)
        return features, vectorizer
    else: 
        features = vectorizer.transform(text)
        return features



def evaluate_model(model, X_test, y_test):

    y_pred = model.predict(X_test)

    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Accuracy Score: {:.2f}".format((accuracy_score(y_test, y_pred))))    
    print("Precision Score: {:.2f}".format((precision_score(y_test, y_pred))))
    print("Recall Score: {:.2f}".format((recall_score(y_test, y_pred))))
    print("F1 Score: {:.2f}".format((f1_score(y_test, y_pred))))



#############


def test_loop(model, test_loader):
    # define the test loop
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    # move the model to the device
    model.to(device)
    model.eval()
    test_preds = []
    test_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch[0]["input_ids"].to(device)
            attention_mask = batch[0]["attention_mask"].to(device)
            labels = batch[1].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            test_preds.extend(torch.argmax(outputs.logits, axis=1).tolist())
            test_labels.extend(labels.tolist())
    
    # calculate the metrics and print the results
    acc = accuracy_score(test_labels, test_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(test_labels, test_preds, average="binary")
    print(f"Test Accuracy = {acc:.4f}")
    print(f"Test Precision = {precision:.4f}")
    print(f"Test Recall = {recall:.4f}")
    print(f"Test F1 Score = {f1:.4f}")
    print("Classification Report:\n", classification_report(test_labels, test_preds))
    print("Confusion Matrix:\n", confusion_matrix(test_labels, test_preds))

# dataset class
class SentimentDataset(Dataset):
    def __init__(self, encoded_data, labels):
        self.encoded_data = encoded_data
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {key: torch.tensor(text[idx]) for key, text in self.encoded_data.items()}, torch.tensor(self.labels[idx])

#Load TOP 3 Models


In [ ]:
# Define the file name for the pickle file
model_path_TFIDF_SVM = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','TFIDF_SVM.pkl')
model_path_TFIDF_MLP = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','TFIDF_MLP.pkl')
Feature_extractor_path = os.path.join( GOOGLE_DRIVE_PATH,'feature_models','TFIDF.tsv')

# Load the  model from the pickle file
with open(model_path_TFIDF_SVM, 'rb') as file:
    TFIDF_SVM = pickle.load(file)


# Load the  model from the pickle file
with open(model_path_TFIDF_MLP, 'rb') as file:
    TFIDF_MLP = pickle.load(file)


# Load feature extractor from a file
with open(Feature_extractor_path, 'rb') as f:
    feat_extractor = pickle.load(f)

###############

model_path_bert = os.path.join( GOOGLE_DRIVE_PATH,'trained_models','DISTILBERT.pth')
BERT = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# move the model to the device
BERT.to(device)

# Load the model state dictionary from the saved file
BERT.load_state_dict(torch.load(model_path_bert,map_location=device))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

<All keys matched successfully>

# TEST MODEL FUNCTION

In [ ]:
def Testmodel(dataset_path, model_type):
    """
    
    Args:
    dataset_path (str): Path to the test set of the provided dataset.
    model_type (str): String used to select a given model.
    
     where model_type can be one of:
       ['TFIDF_SVM','TFIDF_MLP','BERT']
      
    """
    # Define available model types
    available_model_types = ['TFIDF_SVM','TFIDF_MLP','BERT']

    # Check if provided model type is valid
    if model_type not in available_model_types:
        print("Invalid model type. Available model types are:")
        for item in available_model_types:
            print(item)
        return

    if model_type == 'TFIDF_SVM':
      model = TFIDF_SVM

    elif model_type =='TFIDF_MLP':
      model = TFIDF_MLP


    if model_type == 'TFIDF_MLP'or model_type =='TFIDF_SVM':

      # load data 
      dataset = pd.read_csv(dataset_path, sep='\t')

      #Extract input and output data
      X_test = dataset['review']
      y_test = dataset['label']

      X_test = custom_feature_extraction(X_test, compute=0, vectorizer=feat_extractor)

      #Predict & Evaluate 
      evaluate_model(model, X_test, y_test)



    if model_type == 'BERT':
      model = BERT
      # load data 
      dataset = pd.read_csv(dataset_path, sep='\t')

      #Extract input and output data
      X_test = dataset['review']
      y_test = dataset['label']

      test_encoded_data = tokenizer(X_test.tolist(), padding=True, truncation=True)
      test_data = SentimentDataset({key: text for key, text in test_encoded_data.items()}, y_test.tolist())
      test_loader = DataLoader(test_data, batch_size=24, shuffle=False)


      # test the model
      test_loop(BERT, test_loader)




In [ ]:

# Function to test top 3  models 

# Args:
# dataset_path (str): Path to the test set of the provided dataset.
# model_type (str): String used to select a given model.

#  where model_type can be one of
#    - TFIDF_MLP,
#    - TFIDF_SVM
#    - BERT    #Make sure GPU is available
dataset_path = os.path.join( GOOGLE_DRIVE_PATH,'dataset','test_data.tsv')
Testmodel(dataset_path,'TFIDF_SVM')


Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.88      0.87      1951
           1       0.88      0.87      0.88      2049

    accuracy                           0.88      4000
   macro avg       0.87      0.88      0.87      4000
weighted avg       0.88      0.88      0.88      4000

Confusion Matrix:
 [[1718  233]
 [ 267 1782]]
Accuracy Score: 0.88
Precision Score: 0.88
Recall Score: 0.87
F1 Score: 0.88
